# Sustainable Agriculture - Tillage
_____

The purpose of this notebook is to demonstrate how the Descartes Labs platform can assist in quickly developing sustainable agriculture analysis and workflows. 

We'll look at tillage indices from the literature, which can be quickly derived and viewed on an interactive map. Ground truth would be needed to correlate these indices with meaningful values.

You can run the cells in this notebook one at a time by using `Shift-Enter`

In [1]:
# keep logging quiet
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.captureWarnings(True)

In [2]:
# import packages
import descarteslabs as dl
import descarteslabs.workflows as wf

import ipywidgets
import ipyleaflet
from ipyleaflet import GeoData

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

### Estimating Crop Residue / Tillage

Next, we will explore Crop Residue Indices (CRIs). The indices are derived from Sentinel-2 satellite imagery, and have been found to be correlated with tillage practice and crop residue cover. 

Using Descartes Labs `Workflows`, we get an image collection in Iowa at the beginning of the growing season. We then derive CRIs from the spectral information in Sentinel-2 imagery, and display the indices and Sentinel-2 imagery on an interactive map. 

In [3]:
# Set up the interactive map
m = wf.map
# m.center = (42.344841, -93.168481)  # Iowa South Fork Watershed
m.center = (41.7346976,-95.0216351)
m.zoom = 13

#### Define a Sentinel-2 Image Collection. 
Following the recommendations in the paper "Estimates of Conservation Tillage Practices Using Landsat Archive", we filter out clouds, high NDVI values, and invalid pixels.

Paper link: https://www.mdpi.com/2072-4292/12/16/2665

In [4]:
# Set parameters for pulling imagery
start_datetime = "2019-05-01"
end_datetime = "2019-06-01"
product_id = "sentinel-2:L1C"
cloud_band = "valid_cloudfree"

In [5]:
# Define the Sentinel-2 Image Collection 
ic = wf.ImageCollection.from_id(product_id, start_datetime, end_datetime)

# Mask by clouds
cloudmask = (
    wf.ImageCollection.from_id(
        product_id + ":dlcloud:v1", start_datetime, end_datetime
    ).pick_bands(cloud_band) == 0
)

# Make an ImageCollectionGroupby object, for quicker lookups 
#  from `ic` by date (you can use it like a dict)
ic_date_groupby = ic.groupby(dates=("year", "month", "day"))

# For each cloudmask date, pick the corresponding image from `ic` by date, mosaic both, and mask them.
# (Not all scenes have cloudmasks processed, so this ensures we only return scenes that do.)
masked_ic = cloudmask.groupby(dates=("year", "month", "day")).map(
    lambda ymd, mask_imgs: ic_date_groupby[ymd].mosaic().mask(mask_imgs.mosaic())
)

# Create a S2 median composite layer to display on the map 
masked_ic.pick_bands("red green blue").median(axis="images").visualize("S2 Median Composite")

# Mask by areas where NDVI is greater than 0.3 
red, nir = masked_ic.unpack_bands("red nir")
ndvi = (nir - red)/(nir + red)
masked_ic = masked_ic.mask(ndvi > 0.3)

# Mask out invalid pixels
qa_mask = masked_ic.unpack_bands("alpha")
masked_ic = masked_ic.mask(qa_mask == 1)

# Create a S2 filtered median composite later to display on the map
masked_ic.pick_bands("red green blue").median(axis="images").visualize("S2 Median Composite, filtered")

#### Compute relevant Crop Residue Indices

We compute indices that are outlined in the following paper, titled "A Comparison of Estimating Crop Residue Cover from Sentinel-2 Data Using Empirical Regressions and Machine Learning Methods": https://www.mdpi.com/2072-4292/12/9/1470

In [6]:
# Get relevant bands for the Crop Residue Indices (CRIs)
b2, b3, b4, b8a, b11, b12 = masked_ic.unpack_bands("blue green red red-edge-4 swir1 swir2")

# Simulated Cellulose Absorption Index (3BI1)
#sim_cai = (100 * (0.5 * (b2 + b12) - b4)).rename_bands("3BI1")
#sim_cai.min(axis="images").visualize("3BI1", scales=[7.35, -0.37], colormap="Greens")

# Simulated Lignin Cellulose Absorption Index (3BI2)
#sim_lcai = ((b2 - b4)/(b2 - b12)).rename_bands("3BI2")
#sim_lcai.min(axis="images").visualize("3BI2", scales=[-0.87, 0.44], colormap="Greens")

# Simulated NDRI (3BI3)
#sim_ndri = ((b12 - b4)/(b12 + b11)).rename_bands("3BI3")
#sim_ndri.min(axis="images").visualize("3BI3", scales=[0.25, 0.00], colormap="Greens")

# Normalized Difference Tillage Index (NDTI)
ndti = ((b11 - b12)/(b11 + b12)).rename_bands("NDTI")
ndti.min(axis="images").visualize("NDTI", scales=[0.08, 0.20], colormap="Greens")

# Normalized Different Residue Index (NDRI)
ndri = ((b4 - b12)/(b4 + b12)).rename_bands('NDRI')
ndri.min(axis="images").visualize("NDRI", scales=[-0.44, -0.06], colormap="Greens")

In [7]:
# Display the map
m


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


### Statistic Aggregation
Lastly, we show an example of how to aggregate county statistics using `Workflows`

In [8]:
places = dl.Places()
iowa_counties = places.prefix('north-america_united-states_iowa')['features']

geoms = []
names = []
for c in iowa_counties:
    geoms.append(Polygon(c['geometry']['coordinates'][0]))
    names.append(c['properties']['name'])

d = {'county': names, 'geometry': geoms}
gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")
gdf.head()

,county,geometry
0,Boone,"POLYGON ((-94.164619 42.0346, -94.164704 42.20..."
1,Dallas,"POLYGON ((-94.280496 41.688326, -94.28034 41.8..."
2,Grundy,"POLYGON ((-93.00290200000001 42.470638, -93.02..."
3,Hamilton,"POLYGON ((-93.815729 42.209597, -93.9315809999..."
4,Hardin,"POLYGON ((-93.144583 42.556742, -93.027 42.556..."


### Show the geojson features on the map

In [9]:
# Add layer control
layer_control = ipyleaflet.LayersControl(position="topright")
m.add_control(layer_control)

# Set up widget for metadata display
output = ipywidgets.Output(
    layout={'min_width':'150px','min_height':'20px',
            'max_width':'500px','max_height':'200px'}
)
output = wf.interactive.clearable.ClearableOutput(output)
output = output.children[0]
output_ctrl = ipyleaflet.WidgetControl(widget=output, position='bottomright')
m.add_control(output_ctrl)
@output.capture()

# Set up hovering feature
def circle_hover(feature, **kwargs):
    output.clear_output()
    print(f"{feature['properties']['county']} County")
    
with output:
    # Add metadata to the map from a geodataframe
    geodata = gdf.__geo_interface__
    circle_marker = ipyleaflet.GeoJSON(
        data=geodata,
        hover_style={"fillColor": "#2E99DF", "fillOpacity": 0.7},
        name="Iowa Counties"
    )
    circle_marker.on_hover(circle_hover)
    m.add_layer(circle_marker)
    
# Reset map to see counties
m.center = (42.1891, -93.1264)
m.zoom = 13

### Convert the polygons into geocontexts that are compatible with workflows
You may want to change the parameters, for example to a different resolution

In [10]:
def create_wf_geocontext(geom):
    ctx = wf.GeoContext(
        geometry=geom,
        crs='EPSG:3857',
        resolution=100.
    )
    return ctx

geocontexts = [create_wf_geocontext(g.buffer(0)) for g in gdf.geometry]

### Aggregation over the NDRI and NDTI indices

In [11]:
# Define final portion of workflow that aggregates the indices
ndri_ = ndri.min(axis="images").mean(axis='pixels')
ndti_ = ndti.min(axis="images").mean(axis='pixels')

# Compute aggregated statistics for the first five counties
agg = [[ndri_.compute(ctx)['NDRI'], ndti_.compute(ctx)['NDTI']] for ctx in geocontexts[:5]]

# Put the outputs into a dataframe
df = pd.DataFrame(agg, columns=['NDRI', 'NDTI'])
agg_values = gdf.join(df)
agg_values.head()


Job ID: 987430251c785bc965d283160c75ce39c9070691c9ebd77a
[######] | Steps: 512/512 | Stage: SUCCEEDED                                  
Job ID: 670f39a2a6332ee69db14a28a46072031ab65b319e957449
[######] | Steps: 439/439 | Stage: SUCCEEDED                                  
Job ID: 8354785cf24c201d1f827313b3929c0f3ac5a3f442189b28
[######] | Steps: 567/567 | Stage: SUCCEEDED                                  
Job ID: 03c88879a0ae04e6ed12ca60e4b4e9ed88d3320189469ab3
[######] | Steps: 598/598 | Stage: SUCCEEDED                                  
Job ID: 7d121c7bc0aafd7c9df63572bb7869504719dd7d3be497c9
[######] | Steps: 1/1 | Stage: SUCCEEDED                                      
Job ID: 980c0faae8aeced1fc4dc3258ae055c2c6f205c2a5143bef
[######] | Steps: 260/260 | Stage: SUCCEEDED                                  
Job ID: e84fe08f50726e5efccec393a9b13b3062765c0d56211231
[######] | Steps: 348/348 | Stage: SUCCEEDED                                  
Job ID: 3f37cecbbdfa9726d1db802859bbdf399b8941f

,county,geometry,NDRI,NDTI
0,Boone,"POLYGON ((-94.164619 42.0346, -94.164704 42.20...",-0.367750,0.097197
1,Dallas,"POLYGON ((-94.280496 41.688326, -94.28034 41.8...",-0.342766,0.105041
2,Grundy,"POLYGON ((-93.00290200000001 42.470638, -93.02...",-0.274443,0.153745
3,Hamilton,"POLYGON ((-93.815729 42.209597, -93.9315809999...",-0.357089,0.100197
4,Hardin,"POLYGON ((-93.144583 42.556742, -93.027 42.556...",-0.298735,0.129976
